In [1]:
import pandas as pd
import numpy as np
import pickle
from preprocess_text import *
from bigrams import *
from smoothers import *
from LanguageModels import *

In [2]:
df = pd.read_csv('A1_dataset.csv')

In [3]:
# text = df['TEXT'].to_list()
# def preprocess_text(text):
#     text = lowercase_text(text)
#     text = remove_url_html(text)
#     text = remove_users(text)
#     text = remove_punctuations(text)
#     text = remove_whitespaces(text)
#     text = tokenization(text)
#     text = spelling_correction(text, 'autocorrect_full')
#     text = remove_alphanum(text)
#     return text
# df['preprocessed_text'] = df['TEXT'].progress_apply(preprocess_text)
# preprocess_text = df['preprocessed_text'].to_list()
# df.to_csv('preprocessed_A1.csv', index=False)

In [4]:
df = pd.read_csv('preprocessed_A1.csv')
preprocess_text = df['preprocessed_text'].to_list()
for i in range(len(preprocess_text)):
    preprocess_text[i] = eval(preprocess_text[i])
    

In [5]:
unigram_counts = {}
for sentence in preprocess_text:
    for word in sentence:
        if word in unigram_counts:
            unigram_counts[word] += 1
        else:
            unigram_counts[word] = 1

In [6]:
unigram_probs = {}
scale_factor = sum(unigram_counts.values())
for word in unigram_counts:
    unigram_probs[word] = unigram_counts[word] / scale_factor

In [7]:
len(unigram_counts)

6952

# Saving Bigrams with Smoothing

In [7]:
# bigram_wrapper = Bigrams(preprocess_text)
# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

# with open('bigram_model_dumps/bigram_wrapper.pickle', 'wb') as f:
#     pickle.dump(bigram_wrapper, f)

In [9]:
# load bigram_wrapper
# with open('bigram_wrapper.pickle', 'rb') as f:
#     bigram_wrapper = pickle.load(f)

# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

In [10]:
# smoother = LMSmoothers(bigrams, vocab_len, unigram_counts=unigram_counts, unigrams_probs=unigram_probs)
# laplace_smoothed_bigrams = smoother.laplace()
# add_5_smoothed_bigrams = smoother.add_k(5)
# add_5_with_unigram_prior_smoothed_bigrams = smoother.add_k_with_unigram_prior(5)

In [11]:
# with open('bigram_model_dumps/laplace_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(laplace_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('bigram_model_dumps/add_5_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('bigram_model_dumps/add_5_with_unigram_prior_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_with_unigram_prior_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Creating BiGram Models

In [8]:
with open('bigram_model_dumps/laplace_smoothed_bigrams.pickle', 'rb') as handle:
    laplace_smoothed_bigrams = pickle.load(handle)

# with open('bigram_model_dumps/add_5_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_smoothed_bigrams = pickle.load(handle)

# with open('bigram_model_dumps/add_5_with_unigram_prior_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_with_unigram_prior_smoothed_bigrams = pickle.load(handle)

In [10]:
# Load Negative Prompts
with open('Pos Neg Prompts/neg_250_vader.txt', 'rb') as handle:
    neg_prompts = handle.readlines()

for i in range(len(neg_prompts)):
    neg_prompts[i] = neg_prompts[i].decode('utf-8').strip()

In [ ]:
run_config = {
    'save_path': 'neg_ppl_normalized.txt',
    'bigrams': laplace_smoothed_bigrams,
    'unigram_probs': unigram_probs,
    'sentiment_model': 'vader',
    'add_to_numerator': False,
    'add_to_denominator': False,
    'add_externally': True,
    'normalize_by_perplexity': True,
    'sentiment_scale_factor': 1,
}

In [12]:
# clear the file
open('neg_ppl_normalized.txt', 'w').close()

In [13]:
def append_to_file(filename, text1, text2):
    with open(filename, 'a') as f:
        f.write(text1 + " " + text2 + '\n')

In [14]:
ls_neg_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2, normalizebyPerplexity=True, sentiment_scale_factor=1)
for p in neg_prompts:
    sentence_neg = lm.generate_text([p], sentiment=-1, length=10)
    print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_neg)
    print("Negative Perplexity: {}".format(ppl))
    ls_neg_gen.append((sentence_neg, ppl))
    append_to_file('neg_ppl_normalized.txt', " ".join(sentence_neg), str(ppl))

Negative: ['raping', 'slavery', 'killed', 'the', 'shit', 'alone', 'evil', 'and', 'crisis', 'i']
Negative Perplexity: 4233.083434158437
Negative: ['slavery', 'hell', 'is', 'killing', 'me', 'kill', 'raping', 'tragedy', 'utter', 'shittiest']
Negative Perplexity: 4600.875501584128
Negative: ['kill', 'me', 'cry', 'why', 'do', 'killed', 'a', 'bad', 'killing', 'me']
Negative Perplexity: 2720.146253612938
Negative: ['hell', 'everybody', 'tonight', 'slavery', 'raping', 'horrific', 'killed', 'a', 'killing', 'me']
Negative Perplexity: 4793.550880054558
Negative: ['killed', 'raping', 'slavery', 'tragedy', 'kill', 'hell', 'evil', 'is', 'killing', 'me']
Negative Perplexity: 4902.756700714179


In [20]:
# save generated sentences
with open('ls_neg_gen_vader_thread.pickle', 'wb') as handle:
    pickle.dump(ls_neg_gen, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
ls_neg_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", True, False, True, repetition_penalty=0.2, sentiment_scale_factor=1e-1)
for p in neg_prompts:
    sentence_neg = lm.generate_text([p], sentiment=-1, length=10)
    print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_neg)
    print("Negative Perplexity: {}".format(ppl))
    ls_neg_gen.append((sentence_neg, ppl))
    append_to_file('neg_gen_add_numerator.txt', " ".join(sentence_neg), str(ppl))

Negative: ['raping', 'slavery', 'kill', 'hell', 'kill', 'kill', 'hell', 'killed', 'slavery', 'slavery']
Negative Perplexity: 6957.298289490414
Negative: ['slavery', 'kill', 'kill', 'killed', 'killed', 'raping', 'raping', 'killed', 'kill', 'slavery']
Negative Perplexity: 6955.899475957435
Negative: ['kill', 'slavery', 'hell', 'kill', 'slavery', 'raping', 'raping', 'hell', 'hell', 'raping']
Negative Perplexity: 6957.597640824956
Negative: ['hell', 'killed', 'killed', 'killed', 'raping', 'hell', 'hell', 'raping', 'slavery', 'slavery']
Negative Perplexity: 6958.597827424857
Negative: ['killed', 'hell', 'killed', 'kill', 'slavery', 'hell', 'hell', 'hell', 'raping', 'kill']
Negative Perplexity: 6959.697729014282
Negative: ['killing', 'hell', 'killed', 'hell', 'hell', 'kill', 'slavery', 'raping', 'kill', 'kill']
Negative Perplexity: 6958.898010682494
Negative: ['evil', 'hell', 'slavery', 'slavery', 'hell', 'raping', 'kill', 'hell', 'raping', 'raping']
Negative Perplexity: 6957.397597763808
Ne

In [21]:
ls_neg_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", True, False, True, repetition_penalty=0.2)
for p in neg_prompts:
    sentence_neg = lm.generate_text([p], sentiment=-1, length=10)
    print(sentence_neg)
    print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_neg)
    print("Negative Perplexity: {}".format(ppl))
    ls_neg_gen.append((sentence_neg, ppl))
    append_to_file('neg_gen_mul_numerator.txt', " ".join(sentence_neg), str(ppl))

['raping', 'slavery', 'and', 'a', 'great', 'time', 'to', 'the', 'best', 'i']
Negative: ['raping', 'slavery', 'and', 'a', 'great', 'time', 'to', 'the', 'best', 'i']
Negative Perplexity: 901.6885266295739
['slavery', 'killed', 'tea', 'so', 'i', 'dont', 'want', 'to', 'the', 'last']
Negative: ['slavery', 'killed', 'tea', 'so', 'i', 'dont', 'want', 'to', 'the', 'last']
Negative Perplexity: 719.3617643744873
['kill', 'me', 'too', 'early', 'today', 'i', 'was', 'just', 'woke', 'up']
Negative: ['kill', 'me', 'too', 'early', 'today', 'i', 'was', 'just', 'woke', 'up']
Negative Perplexity: 761.6359982131581
['hell', 'everybody', 'day', 'to', 'the', 'same', 'page', 'of', 'my', 'phone']
Negative: ['hell', 'everybody', 'day', 'to', 'the', 'same', 'page', 'of', 'my', 'phone']
Negative Perplexity: 1041.171891177359
['killed', 'a', 'good', 'day', 'with', 'you', 'can', 'get', 'my', 'mom']
Negative: ['killed', 'a', 'good', 'day', 'with', 'you', 'can', 'get', 'my', 'mom']
Negative Perplexity: 814.522345506

In [ ]:
ls_neg_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", True, False, True, repetition_penalty=0.2)
for p in neg_prompts:
    sentence_neg = lm.generate_text([p], sentiment=-1, length=10)
    print(sentence_neg)
    print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_neg)
    print("Negative Perplexity: {}".format(ppl))
    ls_neg_gen.append((sentence_neg, ppl))
    append_to_file('neg_gen_div_denominator.txt', " ".join(sentence_neg), str(ppl))